In [3]:

import pandas as pd
import numpy as np
import os

In [5]:
df = pd.read_csv('/Users/ycammarata/Downloads/england/E0 (2).csv')
len(df.columns)

106

In [2]:
df_all = pd.DataFrame()
cols = ['Date', 'FTR', 'HomeTeam', 'AwayTeam', 'FTAG', 'FTHG', 'AS', 'HS', 'AST', 'HST']
for file in os.listdir('/Users/ycammarata/Downloads/england/'):
    try :
        df = pd.read_csv('/Users/ycammarata/Downloads/england/'+file, encoding='latin1', usecols=cols)
        df_all = pd.concat([df_all, df])
    except:
        print('Failed processing : ', file)
df_all.to_csv('england_all.csv')

Failed processing :  E0 (27).csv
Failed processing :  E0 (26).csv
Failed processing :  E0 (23).csv
Failed processing :  E0 (22).csv
Failed processing :  E0 (25).csv
Failed processing :  E0 (28).csv
Failed processing :  E0 (24).csv


In [2]:
import dataset

def create_dataset(input_path, output_filename):
    data_processed = []
    for file in os.listdir(input_path):
        print(file,len(data_processed))
        data_processed += dataset.Dataset(input_path+file).processed_results
   
    """ def map_results(results):
        features = {}

        for result in results:
            for key in result.keys():
                if key not in features:
                    features[key] = []

                features[key].append(result[key])

        for key in features.keys():
            features[key] = np.array(features[key])

        return features, features['result']
 """
    #X, y = map_results(data_processed)
    df = pd.DataFrame.from_dict(data_processed)
    df['result'] = df['result'].apply(lambda x: 2 if x=='A' else 1 if x=='H' else 0)
    df.to_csv(output_filename, index=False)

create_dataset('/Users/ycammarata/Downloads/england_clean/', 'england_data.csv')
#create_dataset('/Users/ycammarata/Downloads/portugal/', 'data/portugal_data.csv')
#create_dataset('/Users/ycammarata/Downloads/netherland/', 'data/netherland_data.csv')
#create_dataset('/Users/ycammarata/Downloads/belgium/', 'data/belgium_data.csv')
#create_dataset('/Users/ycammarata/Downloads/turkey/', 'turkey_data.csv')


E0 (3).csv 0
E0 (11).csv 280
E0 (10).csv 560
E0 (2).csv 839
E0.csv 1119
E0 (9).csv 1188
E0 (17).csv 1467
E0 (5).csv 1746
E0 (4).csv 2026
E0 (16).csv 2306
E0 (8).csv 2583
E0 (15).csv 2863
E0 (19).csv 3142
E0 (7).csv 3421
E0 (6).csv 3701
E0 (18).csv 3981
E0 (14).csv 4261
E0 (1).csv 4539
E0 (13).csv 4817
E0 (12).csv 5096


In [7]:
import datetime
from functools import reduce

class Dataset:
    GAMES_FOR_STATS = 5
    def __init__(self, data):
        self.raw_results = []
        self.processed_results = []

        for idx, row in data.iterrows():
            try:
                row['Date'] = datetime.datetime.strptime(row['Date'], '%d/%m/%Y')
            except:
                try:
                    row['Date'] = datetime.datetime.strptime(row['Date'], '%d/%m/%y')
                except BaseException as err:
                    print(f"*Unexpected {err=}, {type(err)=}"+':'+row)
                    break
            self.raw_results.append(row)


        for result in self.raw_results: 
            try:
                home_statistics = self.get_statistics(result['HomeTeam'], result['Date'])
                if home_statistics is None:
                    continue
    
                away_statistics = self.get_statistics(result['AwayTeam'], result['Date'])
                if away_statistics is None:
                    continue
    
                processed_result = {
                    'result': result['FTR'],
                    'home_team': result['HomeTeam'],
                    'away_team': result['AwayTeam'],
                    'odds_home': float(result['B365H']),
                    'odds_draw': float(result['B365D']),
                    'odds_away': float(result['B365A']),
                }
    
                for label, statistics in [('home', home_statistics), ('away', away_statistics)]:
                    for key in statistics.keys():
                        processed_result[label + '_' + key] = statistics[key]
    
                self.processed_results.append(processed_result)
            except BaseException as err:
                print(f"**Unexpected {err=}, {type(err)=}"+':'+result)
                break
    

    # Filter results to only contain matches played in by a given team, before a given date
    def filter(self, team, date):
        def filter_fn(result):
            return (
                result['HomeTeam'] == team or
                result['AwayTeam'] == team
            ) and (result['Date'] < date)

        return list(filter(filter_fn, self.raw_results))

    # Calculate team statistics
    def get_statistics(self, team, date, matches=GAMES_FOR_STATS):
        recent_results = self.filter(team, date)
        
        if len(recent_results) < matches:
            return None

        # This function maps a result to a set of performance measures roughly scaled between -1 and 1
        def map_fn(result):
            if result['HomeTeam'] == team:
                team_letter, opposition_letter = 'H', 'A'
            else:
                team_letter, opposition_letter = 'A', 'H'

            return {
                'wins': 1 if result['FTR'] == team_letter else 0,
                'draws': 1 if result['FTR'] == 'D' else 0,
                'losses': 1 if result['FTR'] == opposition_letter else 0,
                'goals': int(result['FT{}G'.format(team_letter)]),
                'opposition_goals': int(result['FT{}G'.format(opposition_letter)]),
                'shots': int(result['{}S'.format(team_letter)]),
                'shots_on_target': int(result['{}ST'.format(team_letter)]),
                'opposition_shots': int(result['{}S'.format(opposition_letter)]),
                'opposition_shots_on_target': int(result['{}ST'.format(opposition_letter)]),
            }

        def reduce_fn(x, y):
            result = {}
            for key in x.keys():
                result[key] = x[key] + y[key]
            return result

        return reduce(reduce_fn, map(map_fn, recent_results[-matches:]))

In [8]:
def compute_stats(df):
    teams = df['home_team'].unique()
    teams_stat = pd.DataFrame()
    columns = ['wins', 'draws', 'losses', 'goals',
        'opposition_goals', 'shots', 'shots_on_target',
        'opposition_shots', 'opposition_shots_on_target']

    location = ['home', 'away']

    for team in teams:
        team_dict = {
            'name' : team,
            # should be removed not used
            'odds_home' : df[df['home_team']==team]['odds_home'].mean(),
            'odds_away' : df[df['home_team']==team]['odds_away'].mean(),
            'odds_draw' : df[df['home_team']==team]['odds_draw'].mean()
        }
        for pos in location:
            for col in columns:
                team_dict[pos+'_'+col] = df[df[pos+'_team']==team][pos+'_'+col].max() 

        teams_stat = teams_stat.append(team_dict, ignore_index=True)
    return teams_stat

In [380]:
tmp = pd.read_csv('https://www.football-data.co.uk/mmz4281/2122/F1.csv')
tmp.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,F1,06/08/2021,20:00,Monaco,Nantes,1,1,D,1,1,...,2.02,-1.25,2.02,1.91,2.02,1.91,2.03,1.99,1.97,1.89
1,F1,07/08/2021,16:00,Lyon,Brest,1,1,D,0,1,...,2.76,-1.50,2.00,1.93,2.00,1.93,2.00,1.94,1.96,1.89
2,F1,07/08/2021,20:00,Troyes,Paris SG,1,2,A,1,2,...,2.35,1.50,1.95,1.98,1.93,2.00,2.04,2.00,1.91,1.95
3,F1,08/08/2021,12:00,Rennes,Lens,1,1,D,1,1,...,1.82,-0.50,1.93,2.00,1.93,2.00,1.94,2.00,1.91,1.95
4,F1,08/08/2021,14:00,Bordeaux,Clermont,0,2,A,0,0,...,1.69,0.00,1.87,2.06,1.88,2.06,1.89,2.10,1.84,2.03


In [379]:
df = pd.DataFrame(Dataset(pd.read_csv('https://www.football-data.co.uk/mmz4281/2122/F1.csv')).processed_results)

stats_df = compute_stats(df)
stats_df['league'] = 'France'
stats_df.to_csv('france_stats.csv')

In [9]:
import requests

leagues = [
    {"name": 'France', "url":'https://www.football-data.co.uk/mmz4281/2122/F1.csv'},
    {"name": 'England', "url":'https://www.football-data.co.uk/mmz4281/2122/E0.csv'},
]

for league in leagues:
    df = pd.DataFrame(Dataset(pd.read_csv(league['url'])).processed_results)
    stats_df = compute_stats(df)
    stats_df['league'] = league['name']

    #for idx, row in stats_df.iterrows():
        #tjson = {"team": row.to_json()}
        #response = requests.post("https://dwfootprophet.herokuapp.com/team/add", json=tjson)
        #print(response.json())

cuoco
cuoco
